In [185]:
import numpy as np
import pandas as pd
import matplotlib as mpl
import matplotlib.pyplot as plt
from sklearn.preprocessing import LabelEncoder
from sklearn.model_selection import train_test_split
import tensorflow as tf
import keras 
import os
import json

import random as python_random
import time
from sklearn.metrics import mean_squared_error, r2_score, mean_absolute_percentage_error

from tensorflow.keras.models import Sequential
from tensorflow.keras.layers import LSTM, Dropout, Dense
from tensorflow.keras.callbacks import EarlyStopping, ModelCheckpoint
from tensorflow.keras.metrics import MeanAbsolutePercentageError
from tensorflow.keras.optimizers import Adam
from tensorflow.keras.optimizers import SGD
from tensorflow.keras.optimizers import RMSprop
from tensorflow.keras.optimizers import AdamW
from tensorflow.keras.losses import MeanSquaredError
from tensorflow.keras.metrics import RootMeanSquaredError
from tensorflow.keras.models import load_model

from sklearn.neural_network import MLPRegressor
from sklearn.ensemble import GradientBoostingRegressor
from sklearn.svm import SVR
from sklearn.ensemble import RandomForestRegressor

from keras import layers, models

np.random.seed(123)
python_random.seed(123)
tf.random.set_seed(1234)

from funcionesComunes import *


# Trasnfer Learning

In [227]:
# Leemos los diferentes modelos
for model in os.listdir("models/LSTM"):

    # Creamos un df que contiene los resultados de los modelos freeze de cada especie
    df_Freeze = pd.DataFrame(columns = ["File", "Time", "TrainMSE", "TrainRMSE", "TrainR2", "TrainMAPE", 
                                        "ValidationMSE", "ValidationRMSE", "ValidationR2", "ValidationMAPE",
                                        "TestMSE", "TestRMSE", "TestR2", "TestMAPE"])


    if model != "model_Abies_spectabilis.keras":
        print(f'Procesando el modelo: {model}')

        # Para cada archivo 
        for archivo in os.listdir("RCPMerged"):

            # Comprobamos que el modelo no sea del archivo a procesar
            archivoProcesar = archivo.split("_merged")[0]
            modeloProcesar = model.split("model_")[1].split(".keras")[0]
            print(archivoProcesar, modeloProcesar)

            # Lanzamos la ejecución siempre que no sea el mismo archivo o el global
            if archivoProcesar != modeloProcesar and archivo != "totalMerged.csv":

                # Cargar el archivo CSV con punto y coma como delimitador
                df = pd.read_csv(f'RCPMerged/{archivo}')
                df = df[~df["nametag"].str.startswith("INDI005")]

                # Codificamos, normalización y split de datos
                df = codification(df)
                print(df.shape)

                df, valorNormalizacion = individualNormalization(df)
                print(f"SE HA NORMALIZADO EL ARCHIVO: {archivo}")

                temp_df = df
                train_data, val_data, test_data = split_population_individuals(temp_df, train_pct=0.80, val_pct_in_train=0.20, details=False)
                train_data.shape, val_data.shape, test_data.shape

                # Obtenemos X e y para los datasets de train, val y test 
                WINDOWS_SIZE = 3
                X_train, y_train = df_to_X_y_ind_3(train_data, WINDOWS_SIZE)
                X_val, y_val = df_to_X_y_ind_3(val_data, WINDOWS_SIZE)
                X_test, y_test = df_to_X_y_ind_3(test_data, WINDOWS_SIZE)
                print(X_train.shape, X_test.shape, X_val.shape)

                # Cargamos el modelo global
                modelLSTM = tf.keras.models.load_model(f'models/LSTM/{model}')
                modelLSTMFreeze = modelLSTM

                # Obtener el optimizador del modelo
                optimizer = modelLSTMFreeze.optimizer

                # Obtenemos el valor de batch size
                with open(f'arquitecturaModelosJSON/LSTM/{archivo.split(".csv")[0]}_best_models_updated_no_train.json') as f:
                    parameters = json.load(f)

                batch_size_LSTM = parameters[0]["batch_size"]

                # Indicamos el numero de layers a entrenar
                NUM_TRAINABLE = 1

                numLSTM_layers = sum(1 for layer in modelLSTMFreeze.layers if "lstm" in layer.name)

                numFreezeLayers = numLSTM_layers - NUM_TRAINABLE # Congelamos todas menos la útlima capa

                numberLayersFreezed = 0

                # Congelar las primeras 'numFreezeLayers' capas LSTM
                for i, layer in enumerate(modelLSTMFreeze.layers):

                    if "lstm" in layer.name and numFreezeLayers>numberLayersFreezed:
                        numberLayersFreezed += 1
                        layer.trainable = False

                    print(layer, layer.trainable)

                # Compilamos el modelo con los nuevos datos
                modelLSTMFreeze.compile(optimizer=optimizer, loss=MeanSquaredError(), metrics=[
                                            MeanSquaredError(name='mse'),
                                            RootMeanSquaredError(name='rmse'),
                                            MeanAbsolutePercentageError(name='mape')
                                        ])

                # Comienza a medir el tiempo de entrenamiento
                start_time = time.time()

                historyLSTMTransfer = modelLSTMFreeze.fit(X_train, y_train, epochs=200, batch_size=batch_size_LSTM,
                                        validation_data=(X_val, y_val), callbacks=[EarlyStopping(monitor='val_loss', patience=10)], verbose=0)

                # Finaliza la medición del tiempo de entrenamiento
                end_time = time.time()
                print(f"[MODELO CONGELADO]: {end_time-start_time}")

                # Realizar predicciones y calcular métricas para el conjunto de entrenamiento
                predictions_train = predictionForIndividuals(X_train, y_train, modelLSTMFreeze, batch_size_LSTM)
                predictions_train["PredictionsDenormalize"] = predictions_train.apply(lambda row: desnormalizacionBAI(row, valorNormalizacion, "Predictions"), axis=1)
                predictions_train["ActualDenormalize"] = predictions_train.apply(lambda row: desnormalizacionBAI(row, valorNormalizacion, "Actuals"), axis=1)

                train_mse = mean_squared_error(predictions_train["ActualDenormalize"],predictions_train["PredictionsDenormalize"])
                train_rmse = np.sqrt(train_mse)
                train_mape = (np.sum(np.abs(predictions_train["PredictionsDenormalize"] - predictions_train["ActualDenormalize"])) / np.sum(np.abs(predictions_train["ActualDenormalize"]))) * 100
                train_r2 = r2_score(predictions_train["ActualDenormalize"], predictions_train["PredictionsDenormalize"])

                # Realizar predicciones y calcular métricas para el conjunto de validación
                predictions_val = predictionForIndividuals(X_val, y_val, modelLSTMFreeze, batch_size_LSTM)
                predictions_val["PredictionsDenormalize"] = predictions_val.apply(lambda row: desnormalizacionBAI(row, valorNormalizacion, "Predictions"), axis=1)
                predictions_val["ActualDenormalize"] = predictions_val.apply(lambda row: desnormalizacionBAI(row, valorNormalizacion, "Actuals"), axis=1)

                val_mse = mean_squared_error(predictions_val["ActualDenormalize"],predictions_val["PredictionsDenormalize"])
                val_rmse = np.sqrt(val_mse)
                val_mape = (np.sum(np.abs(predictions_val["PredictionsDenormalize"] - predictions_val["ActualDenormalize"])) / np.sum(np.abs(predictions_val["ActualDenormalize"]))) * 100
                val_r2 = r2_score(predictions_val["ActualDenormalize"], predictions_val["PredictionsDenormalize"])

                # Realizar predicciones y calcular métricas para el conjunto de prueba
                predictions_test = predictionForIndividuals(X_test, y_test, modelLSTMFreeze, batch_size_LSTM)
                predictions_test["PredictionsDenormalize"] = predictions_test.apply(lambda row: desnormalizacionBAI(row, valorNormalizacion, "Predictions"), axis=1)
                predictions_test["ActualDenormalize"] = predictions_test.apply(lambda row: desnormalizacionBAI(row, valorNormalizacion, "Actuals"), axis=1)

                test_mse = mean_squared_error(predictions_test["ActualDenormalize"],predictions_test["PredictionsDenormalize"])
                test_rmse = np.sqrt(test_mse)
                test_mape = (np.sum(np.abs(predictions_test["PredictionsDenormalize"] - predictions_test["ActualDenormalize"])) / np.sum(np.abs(predictions_test["ActualDenormalize"]))) * 100
                test_r2 = r2_score(predictions_test["ActualDenormalize"], predictions_test["PredictionsDenormalize"])

                print(f"RESULTADOS DE MSE, RMSE, R2, MAPE (Train): {train_mse}, {train_rmse}, {train_r2}, {train_mape}")
                print(f"RESULTADOS DE MSE, RMSE, R2, MAPE (Val): {val_mse}, {val_rmse}, {val_r2}, {val_mape}")
                print(f"RESULTADOS DE MSE, RMSE, R2, MAPE (Test): {test_mse}, {test_rmse}, {test_r2}, {test_mape}")

                # Guardamos los datos calculados
                df_Freeze.loc[len(df_Freeze)] = [archivo, end_time-start_time,train_mse, train_rmse, train_r2, train_mape, val_mse, val_rmse, val_r2, val_mape, test_mse, test_rmse, test_r2, test_mape]

    df_Freeze.to_csv(f'resultadosTransfer_{model.split("model_")[1].split(".keras")[0]}.csv', index=False)

Procesando el modelo: model_Pinus gerardiana.keras
Abies spectabilis Pinus gerardiana
(96429, 77)
SE HA NORMALIZADO EL ARCHIVO: Abies spectabilis_merged.csv
(56618, 4, 43) (19637, 4, 43) (15689, 4, 43)
<LSTM name=lstm_106, built=True> False
<Dropout name=dropout_58, built=True> True
<LSTM name=lstm_107, built=True> True
<Dense name=dense_58, built=True> True
<Dropout name=dropout_59, built=True> True
<Dense name=dense_59, built=True> True
[MODELO CONGELADO]: 623.9431483745575


/home/workstation/Escritorio/PabloC/TransferLearning/funcionesComunes.py:191: FutureWarning: The behavior of DataFrame concatenation with empty or all-NA entries is deprecated. In a future version, this will no longer exclude empty or all-NA columns when determining the result dtypes. To retain the old behavior, exclude the relevant entries before the concat operation.
  predictionsDF = pd.concat([predictionsDF, tempDF], ignore_index=True)
/home/workstation/Escritorio/PabloC/TransferLearning/funcionesComunes.py:191: FutureWarning: The behavior of DataFrame concatenation with empty or all-NA entries is deprecated. In a future version, this will no longer exclude empty or all-NA columns when determining the result dtypes. To retain the old behavior, exclude the relevant entries before the concat operation.
  predictionsDF = pd.concat([predictionsDF, tempDF], ignore_index=True)
/home/workstation/Escritorio/PabloC/TransferLearning/funcionesComunes.py:191: FutureWarning: The behavior of Dat

RESULTADOS DE MSE, RMSE, R2, MAPE (Train): 65.76907403315674, 8.109813440095694, 0.42297784389515736, 43.48150028966914
RESULTADOS DE MSE, RMSE, R2, MAPE (Val): 68.62248153780297, 8.28386875425987, 0.3747363004208396, 44.29725816992263
RESULTADOS DE MSE, RMSE, R2, MAPE (Test): 70.56011836836414, 8.400007045733005, 0.47670810330411184, 42.90733950893359
Picea smithiana Pinus gerardiana
(34476, 77)
SE HA NORMALIZADO EL ARCHIVO: Picea smithiana_merged.csv
(19933, 4, 43) (7272, 4, 43) (5863, 4, 43)
<LSTM name=lstm_106, built=True> False
<Dropout name=dropout_58, built=True> True
<LSTM name=lstm_107, built=True> True
<Dense name=dense_58, built=True> True
<Dropout name=dropout_59, built=True> True
<Dense name=dense_59, built=True> True
[MODELO CONGELADO]: 312.68973088264465


/home/workstation/Escritorio/PabloC/TransferLearning/funcionesComunes.py:191: FutureWarning: The behavior of DataFrame concatenation with empty or all-NA entries is deprecated. In a future version, this will no longer exclude empty or all-NA columns when determining the result dtypes. To retain the old behavior, exclude the relevant entries before the concat operation.
  predictionsDF = pd.concat([predictionsDF, tempDF], ignore_index=True)
/home/workstation/Escritorio/PabloC/TransferLearning/funcionesComunes.py:191: FutureWarning: The behavior of DataFrame concatenation with empty or all-NA entries is deprecated. In a future version, this will no longer exclude empty or all-NA columns when determining the result dtypes. To retain the old behavior, exclude the relevant entries before the concat operation.
  predictionsDF = pd.concat([predictionsDF, tempDF], ignore_index=True)
/home/workstation/Escritorio/PabloC/TransferLearning/funcionesComunes.py:191: FutureWarning: The behavior of Dat

RESULTADOS DE MSE, RMSE, R2, MAPE (Train): 182.3452376973931, 13.503526861431167, 0.4656238342728847, 39.39238511446368
RESULTADOS DE MSE, RMSE, R2, MAPE (Val): 208.52884703400912, 14.440527934740098, 0.3982410750781795, 42.191203211013146
RESULTADOS DE MSE, RMSE, R2, MAPE (Test): 214.18067729206908, 14.634912958130945, 0.276721046383806, 44.374605342784655
Abies pindrow Pinus gerardiana
(16018, 77)
SE HA NORMALIZADO EL ARCHIVO: Abies pindrow_merged.csv
(9042, 4, 43) (3441, 4, 43) (2815, 4, 43)
<LSTM name=lstm_106, built=True> False
<Dropout name=dropout_58, built=True> True
<LSTM name=lstm_107, built=True> True
<Dense name=dense_58, built=True> True
<Dropout name=dropout_59, built=True> True
<Dense name=dense_59, built=True> True
[MODELO CONGELADO]: 50.32997393608093


...

RESULTADOS DE MSE, RMSE, R2, MAPE (Train): 4.702693471014114, 2.168569452660927, 0.6248045758711112, 39.75516582471016
RESULTADOS DE MSE, RMSE, R2, MAPE (Val): 1.5344546525270881, 1.2387310654565373, 0.31912661293165, 43.213953786867386
RESULTADOS DE MSE, RMSE, R2, MAPE (Test): 14.058667375992066, 3.749488948642477, 0.2583970390191178, 54.17569406315532
Juniperus recurva Picea smithiana
(5316, 77)
SE HA NORMALIZADO EL ARCHIVO: Juniperus recurva_merged.csv
(3188, 4, 43) (1082, 4, 43) (810, 4, 43)
<LSTM name=lstm_85, built=True> False
<LSTM name=lstm_86, built=True> False
<Dropout name=dropout_46, built=True> True
<LSTM name=lstm_87, built=True> True
<Dense name=dense_46, built=True> True
<Dropout name=dropout_47, built=True> True
<Dense name=dense_47, built=True> True
[MODELO CONGELADO]: 52.57010197639465


/home/workstation/Escritorio/PabloC/TransferLearning/funcionesComunes.py:191: FutureWarning: The behavior of DataFrame concatenation with empty or all-NA entries is deprecated. In a future version, this will no longer exclude empty or all-NA columns when determining the result dtypes. To retain the old behavior, exclude the relevant entries before the concat operation.
  predictionsDF = pd.concat([predictionsDF, tempDF], ignore_index=True)
/home/workstation/Escritorio/PabloC/TransferLearning/funcionesComunes.py:191: FutureWarning: The behavior of DataFrame concatenation with empty or all-NA entries is deprecated. In a future version, this will no longer exclude empty or all-NA columns when determining the result dtypes. To retain the old behavior, exclude the relevant entries before the concat operation.
  predictionsDF = pd.concat([predictionsDF, tempDF], ignore_index=True)
/home/workstation/Escritorio/PabloC/TransferLearning/funcionesComunes.py:191: FutureWarning: The behavior of Dat

RESULTADOS DE MSE, RMSE, R2, MAPE (Train): 10.395504752761099, 3.2242060654928832, 0.5869210797971465, 36.795089619871646
RESULTADOS DE MSE, RMSE, R2, MAPE (Val): 7.7610851074987455, 2.7858724140740447, 0.7181840065225334, 30.11552917355952
RESULTADOS DE MSE, RMSE, R2, MAPE (Test): 3.0686001242675327, 1.7517420256040936, 0.6737702194127206, 27.0388024396503
totalMerged.csv Picea smithiana
Juniperus spp.  Picea smithiana
(17976, 77)
SE HA NORMALIZADO EL ARCHIVO: Juniperus spp. _merged.csv
(10218, 4, 43) (3900, 4, 43) (3018, 4, 43)
<LSTM name=lstm_85, built=True> False
<LSTM name=lstm_86, built=True> False
<Dropout name=dropout_46, built=True> True
<LSTM name=lstm_87, built=True> True
<Dense name=dense_46, built=True> True
<Dropout name=dropout_47, built=True> True
<Dense name=dense_47, built=True> True
[MODELO CONGELADO]: 77.21886897087097


/home/workstation/Escritorio/PabloC/TransferLearning/funcionesComunes.py:191: FutureWarning: The behavior of DataFrame concatenation with empty or all-NA entries is deprecated. In a future version, this will no longer exclude empty or all-NA columns when determining the result dtypes. To retain the old behavior, exclude the relevant entries before the concat operation.
  predictionsDF = pd.concat([predictionsDF, tempDF], ignore_index=True)
/home/workstation/Escritorio/PabloC/TransferLearning/funcionesComunes.py:191: FutureWarning: The behavior of DataFrame concatenation with empty or all-NA entries is deprecated. In a future version, this will no longer exclude empty or all-NA columns when determining the result dtypes. To retain the old behavior, exclude the relevant entries before the concat operation.
  predictionsDF = pd.concat([predictionsDF, tempDF], ignore_index=True)
/home/workstation/Escritorio/PabloC/TransferLearning/funcionesComunes.py:191: FutureWarning: The behavior of Dat

RESULTADOS DE MSE, RMSE, R2, MAPE (Train): 32.95594840390553, 5.740727166823514, 0.4625110615240532, 41.27119119289308
RESULTADOS DE MSE, RMSE, R2, MAPE (Val): 9.284263489402365, 3.047008941470695, 0.5280645248606545, 41.503350510372044
RESULTADOS DE MSE, RMSE, R2, MAPE (Test): 51.28090274538589, 7.161068547736845, 0.5092563888650669, 42.62590826817508


# Entrenamiento desde 0 de los modelos

In [186]:
# Función para crear el modelo LSTM basado en los parámetros del JSON
def create_lstm_model(params, input_shape):
    model = tf.keras.Sequential()

    # Añadir capas LSTM
    for i in range(params['num_lstm_layers']):
        if i == 0:
            # Primera capa LSTM necesita el input_shape
            model.add(tf.keras.layers.LSTM(units=params[f'units_lstm_{i}'],
                                           return_sequences=True,
                                           use_bias = True, 
                                           input_shape=input_shape))
        else:
            # Capas posteriores
            model.add(tf.keras.layers.LSTM(units=params[f'units_lstm_{i}'], 
                                           use_bias = True,
                                           return_sequences=True))

    # CAPA DROPOUT
    model.add(Dropout(params["dropout"]))

    # Añadir las capas LSTM adicionales (si existen)
    for i in range(params['num_lstm_layers_after']):
        model.add(tf.keras.layers.LSTM(units=params[f'units_lstm_after_{i}'], 
                                       use_bias = True,
                                       return_sequences=(i < params['num_lstm_layers_after'] - 1)))

    # Añadir capa densa
    model.add(tf.keras.layers.Dense(units=params['dense_units'], activation=params['dense_activation'], use_bias = True))

    # DROPOUT 2
    model.add(Dropout(params["dropout_2"]))

    # Añadir la capa de salida (una sola unidad de salida)
    model.add(tf.keras.layers.Dense(1, activation=params["dense_activation"], use_bias = True))

    # Compilar el modelo según el optimizador especificado en los parámetros
    learning_rate = params['learning_rate']
    optimizer_name = params['optimizer']

    if optimizer_name == 'rmsprop':
        optimizer = RMSprop(learning_rate=learning_rate)
    elif optimizer_name == 'adam':
        optimizer = Adam(learning_rate=learning_rate)
    elif optimizer_name == 'sgd':
        optimizer = SGD(learning_rate=learning_rate)
    else:
        raise ValueError(f"Optimizer {optimizer_name} not recognized")

    # Compilamos el modelo
    model.compile(optimizer=optimizer,
                  loss='mean_squared_error',  # Asumo que quieres minimizar el MSE
                  metrics=['mse', 'mae', tf.keras.metrics.RootMeanSquaredError(name='rmse')])

    return model

In [210]:
df_Not_Freeze = pd.DataFrame(columns = ["File", "Time", "TrainMSE", "TrainRMSE", "TrainR2", "TrainMAPE", 
                                "ValidationMSE", "ValidationRMSE", "ValidationR2", "ValidationMAPE",
                                "TestMSE", "TestRMSE", "TestR2", "TestMAPE"])

for archivo in os.listdir("RCPMerged"):

    if archivo != "totalMerged.csv" and archivo != "Abies spectabilis_merged.csv":

        # Cargar el archivo CSV con punto y coma como delimitador
        df = pd.read_csv(f'RCPMerged/{archivo}')
        df = df[~df["nametag"].str.startswith("INDI005")]

        # Codificamos, normalización y split de datos
        df = codification(df)
        print(df.shape)

        df, valorNormalizacion = individualNormalization(df)
        print(f'SE HA NORMALIZADO EL ARCHIVO: {archivo.replace("_"," ")}')

        temp_df = df
        train_data, val_data, test_data = split_population_individuals(temp_df, train_pct=0.80, val_pct_in_train=0.20, details=False)
        train_data.shape, val_data.shape, test_data.shape

        # Obtenemos X e y para los datasets de train, val y test 
        WINDOWS_SIZE = 3
        X_train, y_train = df_to_X_y_ind_3(train_data, WINDOWS_SIZE)
        X_val, y_val = df_to_X_y_ind_3(val_data, WINDOWS_SIZE)
        X_test, y_test = df_to_X_y_ind_3(test_data, WINDOWS_SIZE)
        print(X_train.shape, X_test.shape, X_val.shape)

        # Lectura de la arquitectura del modelo LSTM
        with open(f'arquitecturaModelosJSON/LSTM/{archivo.split(".csv")[0]}_best_models_updated_no_train.json', "r") as f:
            arquitectura_json = json.load(f)

        arquitecturaLSTM = arquitectura_json[0]

        # Crear el modelo desde cero basado en las configuraciones
        input_shape = (X_train.shape[1], X_train.shape[2])
        modelLSTM = create_lstm_model(arquitecturaLSTM, input_shape)

        # Obtener el optimizador y otros hiperparámetros desde el JSON
        learning_rate = arquitecturaLSTM['learning_rate']
        optimizer_name = arquitecturaLSTM['optimizer']
        batch_size_LSTM = arquitecturaLSTM['batch_size']

        if optimizer_name == 'rmsprop':
            opt = RMSprop(learning_rate=learning_rate)
        elif optimizer_name == 'adam':
            opt = Adam(learning_rate=learning_rate)
        elif optimizer_name == 'sgd':
            opt = SGD(learning_rate=learning_rate)
        else:
            raise ValueError(f"Optimizador {optimizer_name} no reconocido")

        # Compilar el modelo con los hiperparámetros del JSON
        modelLSTM.compile(optimizer=opt,
                    loss=MeanSquaredError(),
                    metrics=[
                        MeanSquaredError(name='mse'),
                        RootMeanSquaredError(name='rmse'),
                        MeanAbsolutePercentageError(name='mape')
                    ])
        
        # Inicio del entrenamiento 
        start_time = time.time()

        # Entrenar el modelo
        historyLSTM = modelLSTM.fit(X_train, y_train, epochs=200, batch_size=batch_size_LSTM,
                    validation_data=(X_val, y_val), callbacks=[EarlyStopping(monitor='val_loss', patience=20)], verbose = 0)
        
        # Finaliza la medición del tiempo de entrenamiento
        end_time = time.time()
        print(f"[MODELO CONGELADO]: {end_time-start_time}")

        # Guardamos el modelo entrenado 
        modelLSTM.save(f'models/model_{archivo.split("_merged")[0]}.keras')

        # Realizar predicciones y calcular métricas para el conjunto de entrenamiento
        predictions_train = predictionForIndividuals(X_train, y_train, modelLSTM, batch_size_LSTM)
        predictions_train["PredictionsDenormalize"] = predictions_train.apply(lambda row: desnormalizacionBAI(row, valorNormalizacion, "Predictions"), axis=1)
        predictions_train["ActualDenormalize"] = predictions_train.apply(lambda row: desnormalizacionBAI(row, valorNormalizacion, "Actuals"), axis=1)

        train_mse = mean_squared_error(predictions_train["ActualDenormalize"],predictions_train["PredictionsDenormalize"])
        train_rmse = np.sqrt(train_mse)
        train_mape = (np.sum(np.abs(predictions_train["PredictionsDenormalize"] - predictions_train["ActualDenormalize"])) / np.sum(np.abs(predictions_train["ActualDenormalize"]))) * 100
        train_r2 = r2_score(predictions_train["ActualDenormalize"], predictions_train["PredictionsDenormalize"])

        # Realizar predicciones y calcular métricas para el conjunto de validación
        predictions_val = predictionForIndividuals(X_val, y_val, modelLSTM, batch_size_LSTM)
        predictions_val["PredictionsDenormalize"] = predictions_val.apply(lambda row: desnormalizacionBAI(row, valorNormalizacion, "Predictions"), axis=1)
        predictions_val["ActualDenormalize"] = predictions_val.apply(lambda row: desnormalizacionBAI(row, valorNormalizacion, "Actuals"), axis=1)

        val_mse = mean_squared_error(predictions_val["ActualDenormalize"],predictions_val["PredictionsDenormalize"])
        val_rmse = np.sqrt(val_mse)
        val_mape = (np.sum(np.abs(predictions_val["PredictionsDenormalize"] - predictions_val["ActualDenormalize"])) / np.sum(np.abs(predictions_val["ActualDenormalize"]))) * 100
        val_r2 = r2_score(predictions_val["ActualDenormalize"], predictions_val["PredictionsDenormalize"])

        # Realizar predicciones y calcular métricas para el conjunto de prueba
        predictions_test = predictionForIndividuals(X_test, y_test, modelLSTM, batch_size_LSTM)
        predictions_test["PredictionsDenormalize"] = predictions_test.apply(lambda row: desnormalizacionBAI(row, valorNormalizacion, "Predictions"), axis=1)
        predictions_test["ActualDenormalize"] = predictions_test.apply(lambda row: desnormalizacionBAI(row, valorNormalizacion, "Actuals"), axis=1)

        test_mse = mean_squared_error(predictions_test["ActualDenormalize"],predictions_test["PredictionsDenormalize"])
        test_rmse = np.sqrt(test_mse)
        test_mape = (np.sum(np.abs(predictions_test["PredictionsDenormalize"] - predictions_test["ActualDenormalize"])) / np.sum(np.abs(predictions_test["ActualDenormalize"]))) * 100
        test_r2 = r2_score(predictions_test["ActualDenormalize"], predictions_test["PredictionsDenormalize"])

        print(f"RESULTADOS DE MSE, RMSE, R2, MAPE (Train): {train_mse}, {train_rmse}, {train_r2}, {train_mape}")
        print(f"RESULTADOS DE MSE, RMSE, R2, MAPE (Val): {val_mse}, {val_rmse}, {val_r2}, {val_mape}")
        print(f"RESULTADOS DE MSE, RMSE, R2, MAPE (Test): {test_mse}, {test_rmse}, {test_r2}, {test_mape}")

        # Guardamos los datos calculados
        df_Not_Freeze.loc[len(df_Not_Freeze)] = [archivo, end_time-start_time,train_mse, train_rmse, train_r2, train_mape, val_mse, val_rmse, val_r2, val_mape, test_mse, test_rmse, test_r2, test_mape]

df_Not_Freeze.to_csv("resultados_Not_Transfer.csv", index=False)


(34476, 77)
SE HA NORMALIZADO EL ARCHIVO: Picea smithiana merged.csv
(19933, 4, 43) (7272, 4, 43) (5863, 4, 43)


/home/workstation/Escritorio/PabloC/TransferLearning/.venv/lib/python3.10/site-packages/keras/src/layers/rnn/rnn.py:204: UserWarning: Do not pass an `input_shape`/`input_dim` argument to a layer. When using Sequential models, prefer using an `Input(shape)` object as the first layer in the model instead.
  super().__init__(**kwargs)


[MODELO CONGELADO]: 576.243658542633


/home/workstation/Escritorio/PabloC/TransferLearning/funcionesComunes.py:191: FutureWarning: The behavior of DataFrame concatenation with empty or all-NA entries is deprecated. In a future version, this will no longer exclude empty or all-NA columns when determining the result dtypes. To retain the old behavior, exclude the relevant entries before the concat operation.
  predictionsDF = pd.concat([predictionsDF, tempDF], ignore_index=True)
/home/workstation/Escritorio/PabloC/TransferLearning/funcionesComunes.py:191: FutureWarning: The behavior of DataFrame concatenation with empty or all-NA entries is deprecated. In a future version, this will no longer exclude empty or all-NA columns when determining the result dtypes. To retain the old behavior, exclude the relevant entries before the concat operation.
  predictionsDF = pd.concat([predictionsDF, tempDF], ignore_index=True)
/home/workstation/Escritorio/PabloC/TransferLearning/funcionesComunes.py:191: FutureWarning: The behavior of Dat

RESULTADOS DE MSE, RMSE, R2, MAPE (Train): 171.35650608905578, 13.090321084261294, 0.49782712259134665, 37.79580734658723
RESULTADOS DE MSE, RMSE, R2, MAPE (Val): 186.241596963849, 13.647036197059382, 0.4625561654478877, 39.92407381916745
RESULTADOS DE MSE, RMSE, R2, MAPE (Test): 203.87084700933804, 14.278334882238125, 0.3115369007042479, 43.19865538799758
(16018, 77)
SE HA NORMALIZADO EL ARCHIVO: Abies pindrow merged.csv
(9042, 4, 43) (3441, 4, 43) (2815, 4, 43)


...

RESULTADOS DE MSE, RMSE, R2, MAPE (Train): 8.45629687955978, 2.9079712652568936, 0.6639780302158466, 33.234290301671074
RESULTADOS DE MSE, RMSE, R2, MAPE (Val): 7.599576065682429, 2.756732860776036, 0.7240486286010984, 29.749938258111857
RESULTADOS DE MSE, RMSE, R2, MAPE (Test): 3.0112565297438, 1.7352972453570599, 0.6798665459140527, 27.580481593978817
(17976, 77)
SE HA NORMALIZADO EL ARCHIVO: Juniperus spp.  merged.csv
(10218, 4, 43) (3900, 4, 43) (3018, 4, 43)


/home/workstation/Escritorio/PabloC/TransferLearning/.venv/lib/python3.10/site-packages/keras/src/layers/rnn/rnn.py:204: UserWarning: Do not pass an `input_shape`/`input_dim` argument to a layer. When using Sequential models, prefer using an `Input(shape)` object as the first layer in the model instead.
  super().__init__(**kwargs)


[MODELO CONGELADO]: 258.2245650291443


/home/workstation/Escritorio/PabloC/TransferLearning/funcionesComunes.py:191: FutureWarning: The behavior of DataFrame concatenation with empty or all-NA entries is deprecated. In a future version, this will no longer exclude empty or all-NA columns when determining the result dtypes. To retain the old behavior, exclude the relevant entries before the concat operation.
  predictionsDF = pd.concat([predictionsDF, tempDF], ignore_index=True)
/home/workstation/Escritorio/PabloC/TransferLearning/funcionesComunes.py:191: FutureWarning: The behavior of DataFrame concatenation with empty or all-NA entries is deprecated. In a future version, this will no longer exclude empty or all-NA columns when determining the result dtypes. To retain the old behavior, exclude the relevant entries before the concat operation.
  predictionsDF = pd.concat([predictionsDF, tempDF], ignore_index=True)
/home/workstation/Escritorio/PabloC/TransferLearning/funcionesComunes.py:191: FutureWarning: The behavior of Dat

RESULTADOS DE MSE, RMSE, R2, MAPE (Train): 20.157254829637722, 4.489683154704541, 0.6712489846085818, 35.417586445858724
RESULTADOS DE MSE, RMSE, R2, MAPE (Val): 8.608813333573943, 2.934077935838437, 0.5623988466501744, 40.58349332693904
RESULTADOS DE MSE, RMSE, R2, MAPE (Test): 30.19989208883748, 5.49544284738159, 0.7109956473826915, 39.595025150939
